In [1]:
from __future__ import division, print_function

import argparse
import copy
from copy import deepcopy
import datetime
import logging
import math
import os
import PIL
import random
import sys
from tqdm import tqdm

import cv2
import numpy as np
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from eval_metrics import ErrorRateAt95Recall
from models import HardNet
from dataset import TripletPhotoTour
from losses import loss_HardNet
from utils import cv2_scale, np_reshape, str2bool

# logging.basicConfig(filename='logs.txt',
#                     filemode='a',
#                     format='%(asctime)s, %(levelname)s: %(message)s',
#                     datefmt='%y-%m-%d %H:%M:%S',
#                     level=logging.DEBUG)
# console = logging.StreamHandler()
# console.setLevel(logging.INFO)
# logging.getLogger().addHandler(console)
logging.basicConfig(level=logging.INFO)

In [2]:
parser = argparse.ArgumentParser(description='PyTorch HardNet')

parser.add_argument('--w1bsroot', type=str,
                    default='data/sets/wxbs-descriptors-benchmark/code/',
                    help='path to dataset')
parser.add_argument('--dataroot', type=str,
                    default='data/',
                    help='path to dataset')
parser.add_argument('--model-dir', default='models/',
                    help='folder to output model checkpoints')
parser.add_argument('--experiment-name', default= 'liberty_train/',
                    help='experiment path')
parser.add_argument('--training-set', default= 'liberty',
                    help='Other options: notredame, yosemite')
parser.add_argument('--loss', default= 'triplet_margin',
                    help='Other options: softmax, contrastive')
parser.add_argument('--batch-reduce', default= 'min',
                    help='Other options: average, random, random_global, L2Net')
parser.add_argument('--num-workers', default= 0, type=int,
                    help='Number of workers to be created')
parser.add_argument('--pin-memory',type=bool, default= True,
                    help='')
parser.add_argument('--anchorave', type=str2bool, default=False,
                    help='anchorave')
parser.add_argument('--imageSize', type=int, default=32,
                    help='the height / width of the input image to network')
parser.add_argument('--mean-image', type=float, default=0.443728476019,
                    help='mean of train dataset for normalization')
parser.add_argument('--std-image', type=float, default=0.20197947209,
                    help='std of train dataset for normalization')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--epochs', type=int, default=10, metavar='E',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--anchorswap', type=str2bool, default=True,
                    help='turns on anchor swap')
parser.add_argument('--batch-size', type=int, default=1024, metavar='BS',
                    help='input batch size for training (default: 1024)')
parser.add_argument('--test-batch-size', type=int, default=1024, metavar='BST',
                    help='input batch size for testing (default: 1024)')
parser.add_argument('--n-triplets', type=int, default=5000000, metavar='N',
                    help='how many triplets will generate from the dataset')
parser.add_argument('--margin', type=float, default=1.0, metavar='MARGIN',
                    help='the margin value for the triplet loss function (default: 1.0')
parser.add_argument('--gor',type=str2bool, default=False,
                    help='use gor')
parser.add_argument('--freq', type=float, default=10.0,
                    help='frequency for cyclic learning rate')
parser.add_argument('--alpha', type=float, default=1.0, metavar='ALPHA',
                    help='gor parameter')
parser.add_argument('--lr', type=float, default=10.0, metavar='LR',
                    help='learning rate (default: 10.0. Yes, ten is not typo)')
parser.add_argument('--fliprot', type=str2bool, default=True,
                    help='turns on flip and 90deg rotation augmentation')
parser.add_argument('--augmentation', type=str2bool, default=False,
                    help='turns on shift and small scale rotation augmentation')
parser.add_argument('--lr-decay', default=1e-6, type=float, metavar='LRD',
                    help='learning rate decay ratio (default: 1e-6')
parser.add_argument('--wd', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)')
# Device options
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--gpu-id', default='0', type=str,
                    help='id(s) for CUDA_VISIBLE_DEVICES')
parser.add_argument('--seed', type=int, default=0, metavar='S',
                    help='random seed (default: 0)')
parser.add_argument('--log-interval', type=int, default=10, metavar='LI',
                    help='how many batches to wait before logging training status')

# args = parser.parse_args()
args = parser.parse_args([
    '--fliprot', 'False',
    '--experiment-name', 'liberty_train/',
    '--gpu-id', '3',
    '--n-triplets', '100000'])

In [3]:
checkpoints = os.path.join('models', f'{args.experiment_name}')

triplet_flag = False

# set the device to use by setting CUDA_VISIBLE_DEVICES env variable in
# order to prevent any memory allocation on unused GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

args.cuda = not args.no_cuda and torch.cuda.is_available()

logging.info(("NOT " if not args.cuda else "") + "Using cuda")

if args.cuda:
    cudnn.benchmark = True
    torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# set random seeds
random.seed(args.seed)
torch.manual_seed(args.seed)
np.random.seed(args.seed)

INFO:root:Using cuda


In [4]:
def train(train_loader, model, optimizer, epoch):
    # switch to train mode
    model.train()
    pbar = tqdm(enumerate(train_loader))
    for batch_idx, data in pbar:

        data_a, data_p = data

        data_a, data_p  = data_a.cuda(), data_p.cuda()
        if args.cuda:
            data_a, data_p  = data_a.cuda(), data_p.cuda()
            out_a = model(data_a)
            out_p = model(data_p)
        else:
            out_a = model(data_a)
            out_p = model(data_p)           

        loss = loss_HardNet(out_a, out_p,
                        margin=args.margin,
                        anchor_swap=args.anchorswap,
                        anchor_ave=args.anchorave,
                        batch_reduce = args.batch_reduce,
                        loss_type = args.loss)
           
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        adjust_learning_rate(optimizer)
        if batch_idx % args.log_interval == 0:
            pbar.set_description('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                                 epoch, batch_idx * len(data_a), len(train_loader.dataset),
                                 100. * batch_idx / len(train_loader),
                                 loss.item()))

    try:
        os.stat(f'{checkpoints}')
    except:
        os.makedirs(f'{checkpoints}')
    
    x = datetime.datetime.now()
    time = x.strftime("%y-%m-%d_%H:%M:%S")
    torch.save({'epoch': epoch + 1, 'state_dict': model.state_dict()}, f'{checkpoints}/checkpoint_{epoch}_{time}.pth')
    logging.info(f'{checkpoints}/checkpoint_{epoch}_{time}.pth')


def test(test_loader, model, epoch):
    # switch to evaluate mode
    model.eval()

    labels, distances = [], []

    pbar = tqdm(enumerate(test_loader))
    for batch_idx, (data_a, data_p, label) in pbar:

        if args.cuda:
            data_a, data_p = data_a.cuda(), data_p.cuda()

        with torch.no_grad():
            out_a = model(data_a)
            out_p = model(data_p)
        dists = torch.sqrt(torch.sum((out_a - out_p) ** 2, 1))  # euclidean distance
        distances.append(dists.data.cpu().numpy().reshape(-1,1))
        ll = label.data.cpu().numpy().reshape(-1, 1)
        labels.append(ll)

        if batch_idx % args.log_interval == 0:
            pbar.set_description(' Test Epoch: {} [{}/{} ({:.0f}%)]'.format(
                epoch, batch_idx * len(data_a), len(test_loader.dataset),
                       100. * batch_idx / len(test_loader)))

    num_tests = test_loader.dataset.matches.size(0)
    labels = np.vstack(labels).reshape(num_tests)
    distances = np.vstack(distances).reshape(num_tests)

    fpr95 = ErrorRateAt95Recall(labels, 1.0 / (distances + 1e-8))
    logging.info('\33[91mTest set: Accuracy(FPR95): {:.8f}\n\33[0m'.format(fpr95))

    return

def adjust_learning_rate(optimizer):
    """Updates the learning rate given the learning rate decay.
    The routine has been implemented according to the original Lua SGD optimizer
    """
    for group in optimizer.param_groups:
        if 'step' not in group:
            group['step'] = 0.
        else:
            group['step'] += 1.
        group['lr'] = args.lr * (
        1.0 - float(group['step']) * float(args.batch_size) / (args.n_triplets * float(args.epochs)))
    return

def create_dataloader(name: str, is_train: bool, load_random_triplet: bool):
    transform = transforms.Compose([
        transforms.Lambda(cv2_scale),
        transforms.Lambda(np_reshape),
        transforms.ToTensor(),
        transforms.Normalize((args.mean_image,), (args.std_image,))])
    
    dataset = TripletPhotoTour(n_triplets=args.n_triplets,
                               fliprot = args.fliprot,
                               train=is_train,
                               load_random_triplets = load_random_triplet,
                               batch_size=args.batch_size,
                               root=args.dataroot,
                               name=name,
                               transform=transform)
    return DataLoader(dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

In [5]:
liberty_dataloader = create_dataloader(name='liberty',
                                       is_train=True,
                                       load_random_triplet=False)
notredame_dataloader = create_dataloader(name='notredame',
                                       is_train=False,
                                       load_random_triplet=False)
yosemite_dataloader = create_dataloader(name='yosemite',
                                       is_train=False,
                                       load_random_triplet=False)

Generating 100000 triplets


100%|██████████| 100000/100000 [00:03<00:00, 26433.60it/s]


In [6]:
model = HardNet()
if args.cuda:
    model = model.cuda()
logging.info('\nparsed options:\n{}\n'.format(vars(args)))

/home/namvh/Workspace/hardnet/code_tmp/models.py:52: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(m.weight.data, gain=0.6)
INFO:root:
parsed options:
{'w1bsroot': 'data/sets/wxbs-descriptors-benchmark/code/', 'dataroot': 'data/', 'model_dir': 'models/', 'experiment_name': 'liberty_train/', 'training_set': 'liberty', 'loss': 'triplet_margin', 'batch_reduce': 'min', 'num_workers': 0, 'pin_memory': True, 'anchorave': False, 'imageSize': 32, 'mean_image': 0.443728476019, 'std_image': 0.20197947209, 'resume': '', 'start_epoch': 0, 'epochs': 10, 'anchorswap': True, 'batch_size': 1024, 'test_batch_size': 1024, 'n_triplets': 100000, 'margin': 1.0, 'gor': False, 'freq': 10.0, 'alpha': 1.0, 'lr': 10.0, 'fliprot': False, 'augmentation': False, 'lr_decay': 1e-06, 'wd': 0.0001, 'no_cuda': False, 'gpu_id': '3', 'seed': 0, 'log_interval': 10, 'cuda': True}



In [ ]:
optimizer = optimizer = optim.SGD(model.features.parameters(), lr=args.lr,
                                  momentum=0.9, dampening=0.9,
                                  weight_decay=args.wd)
# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        logging.info('=> loading checkpoint {}'.format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['state_dict'])
    else:
        logging.info('=> no checkpoint found at {}'.format(args.resume))

In [ ]:
start = args.start_epoch
end = start + args.epochs
for epoch in range(start, end):
    train(liberty_dataloader, model, optimizer, epoch)
    test(notredame_dataloader, model, epoch)
    test(yosemite_dataloader, model, epoch)
    
    liberty_dataloader = create_dataloader(name='liberty',
                                           is_train=True,
                                           load_random_triplet=False)

Train Epoch: 0 [92160/100000 (92%)]	Loss: 0.981797: : 98it [01:37,  1.01it/s]
INFO:root:models/liberty_train//checkpoint_0_20-03-24_11:06:48.pth
 Test Epoch: 0 [30720/100000 (31%)]: : 37it [00:13,  2.72it/s]